In [2]:
import pandas as pd
from datetime import datetime
from meteostat import Point, Daily


df_airports = pd.read_csv('airports.csv') 


inicio = datetime(2015, 1, 1)
fim = datetime(2015, 3, 31)

lista_clima = []

print(f"Iniciando coleta para {len(df_airports)} aeroportos...")

for index, row in df_airports.iterrows():
    iata = row['IATA_CODE']
    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    
    if pd.isna(lat) or pd.isna(lon):
        continue

    try:
        location = Point(lat, lon)
        
        dados = Daily(location, start=inicio, end=fim)
        dados = dados.fetch()
        

        if not dados.empty:

            dados = dados.reset_index()
            
            dados['IATA_CODE'] = iata
            
            cols_to_keep = ['time', 'IATA_CODE', 'tavg', 'tmin', 'tmax', 'prcp', 'snow']

            cols_final = [c for c in cols_to_keep if c in dados.columns]
            
            lista_clima.append(dados[cols_final])
            
    except Exception as e:
        print(f"Erro ao buscar dados para {iata}: {e}")

if lista_clima:
    df_clima_final = pd.concat(lista_clima, ignore_index=True)
    
    df_clima_final.rename(columns={'time': 'DATA'}, inplace=True)
    
    print("Coleta finalizada!")
    print(df_clima_final.head())
    
    df_clima_final.to_csv('clima_aeroportos_2015.csv', index=False)
    print("Arquivo 'clima_aeroportos_2015.csv' salvo.")
else:
    print("Nenhum dado foi coletado.")

Iniciando coleta para 322 aeroportos...


Coleta finalizada!
        DATA IATA_CODE  tavg  tmin  tmax  prcp  snow
0 2015-01-01       ABE  -2.8  -9.9   3.9   0.0   0.0
1 2015-01-02       ABE   0.5  -4.9   5.6   0.0   0.0
2 2015-01-03       ABE  -1.2  -6.6   2.8  16.0   0.0
3 2015-01-04       ABE   4.1   1.7  10.0   4.3   0.0
4 2015-01-05       ABE   0.0  -7.7   7.2   0.0   0.0
Arquivo 'clima_aeroportos_2015.csv' salvo com sucesso.
